# Transfer learning

## 1. Importamos librerias

In [3]:
import os
import time
import copy
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict, List

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms

from torchinfo import summary

## 2. Definimos parámetros

In [2]:
data_dir = './data'
img_size = (300, 300)
bs = 32
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creamos un diccionario vacio para almacenar los resultados
results = {'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

cudnn.benchmark = True
plt.ion()   # modo interactivo

## Definimos el parámetro transform

Vamos a cargar el conjunto de datos y sobre estos datos vamos a aplicar una serie de transformaciones previas:

* Redimensionamos todas las imágenes al mismo tamaño (224x224).
* Aleatoriamente se aplica un giro horizontal de la imagen.
* Rotamos la imagen 10 grados de forma aleatoria.
* Los trasformaremos en tensores y normalizamos $\frac{x-mean}{std}$

Inicialmente, vamos a aplicar la misma transformación al conjunto de datos de entrenamiento que al conjunto de datos de validación

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

### Carga los datos de iNaturalist

In [4]:
image_datasets = {
    x: datasets.INaturalist(
        root = data_dir,
        version = '2021_train_mini' if x == 'train' else '2021_valid',
        transform = data_transforms[x],
        download = False
    )
    for x in ['train', 'val']
}
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x],
        batch_size = bs,
        shuffle = True,
        num_workers = 4
    )
    for x in ['train', 'val']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
inputs, classes = next(iter(dataloaders['train']))

## Definimos la función de entrenamiento

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Etapa {epoch + 1}/{num_epochs}')
        print('-' * 11)

        # Cada iteración tiene una fase de validación y una fase de entrenamiento
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Establece el modelo en modo entrenamiento
            else:
                model.eval()   # Establece el modelo en modo validación

            running_loss = 0.0
            running_corrects = 0

            # Itera sobre los datos
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # pone a cero los gradientes de los tensores optimizados
                optimizer.zero_grad()

                # track historial adelante solo en la fase de entrenamiento
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Solo si está en la fase de entrenamiento retroceder + optimizar
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # estatisticas
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                tittle = 'Entrenamiento'
                results["train_loss"].append(epoch_loss)
                results["train_acc"].append(epoch_acc.item())
            else:
                tittle = 'Validación'
                results["val_loss"].append(epoch_loss)
                results["val_acc"].append(epoch_acc.item())
            print(f'{tittle}\t{(time.time() - since)//60:.0f}min {(time.time() - since)%60:.0f}seg\tPerdida: {epoch_loss:.4f}\tPrecisión: {epoch_acc:.4f}')
            
            # copia del modelo con mejor resultado
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    if time_elapsed < 3600:
        print(f'Entrenamiento completado en: {time_elapsed // 60:.0f}min. {time_elapsed % 60:.0f}seg.')
    else:
        rest_elapsed = time_elapsed % 3600
        print(f'Entrenamiento completado en: {time_elapsed // 3600:.0f}horas {rest_elapsed // 60:.0f}min. {rest_elapsed % 60:.0f}seg.')
    print(f'Mejor precisión validación: {best_acc:4f}')

    # guarda los mejores pesos del modelo
    model.load_state_dict(best_model_wts)
    return model

## ResNet-50 preentrenado

Carga un modelo preentrenado y restablece la última capa totalmente conectada.

In [6]:
model = models.resnet50(weights='IMAGENET1K_V2')
num_ftrs = model.fc.in_features

# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model.fc = nn.Linear(num_ftrs, 10000)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Se optimizan todos los parámetros
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Se decrementa LR por un factor 0.1 cada 7 iteraciones
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Entrenamos el modelo

In [7]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=20)

Etapa 1/20
-----------
Entrenamiento	104min 24seg	Perdida: 7.9920	Precisión: 0.0147
Validación	111min 27seg	Perdida: 5.5578	Precisión: 0.0782

Etapa 2/20
-----------


KeyboardInterrupt: 

Parar aqui/Continuar al terminar el entrenamiento

In [4]:
df_results = pd.DataFrame(results)

NameError: name 'results' is not defined

In [1]:
df_results.to_csv('resultsRestnetV2.txt', sep='\t', index=False)

NameError: name 'df_results' is not defined

In [ ]:
def plot_curves(results: Dict[str, List[float]]):
    
    # Recuperamos los valores de pérdida de entrenamiento y validación
    loss = results['train_loss']
    test_loss = results['val_loss']

    # Recuperamos los valores de precisión de entrenamiento y validación
    accuracy = results['train_acc']
    test_accuracy = results['val_acc']

    # Número de iteraciones que tenemos
    epochs = range(len(results['train_loss']))

    # Definimos el gráfico
    plt.figure(figsize=(15, 7))

    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label='Pérdida entrenamiento')
    plt.plot(epochs, test_loss, label='Pérdida validacion')
    plt.ylabel('Pérdida')
    plt.xlabel('Iteraciones')
    plt.legend()

    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label='Precisión entrenamiento')
    plt.plot(epochs, test_accuracy, label='Precisión validación')
    plt.ylabel('Precisión')
    plt.xlabel('Iteraciones')
    plt.legend();

In [ ]:
plot_curves(results)